In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# uncomment when changes are made to animal_shelter.py to reload the module
# import importlib
# import animal_shelter
# importlib.reload(animal_shelter)

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "Porkchop%3FC%40t%21July7"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H3('Gravioso Salvare Dashboard'))),
    html.H5("Developed by Gregory Anderson", style={'textAlign': 'center', 'marginTop': '10px'}),

    html.A(
        html.Img(src='assets/GraziosoSalvareLogo.png', style={'height': '160px'}),
        href='https://www.snhu.edu',
        target='_blank',
        style={'display': 'block', 'textAlign': 'center', 'marginBottom': '20px'}),
    html.Hr(),
    
    # Interactive Filter Buttons
    html.Div(
        dcc.RadioItems(
            id='rescue-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster/Individual Tracking', 'value': 'tracking'},
                {'label': 'Reset Filters', 'value': 'reset'}
            ],
                 value = 'reset',
                 labelStyle={'display':'inline-block'}
        ),
        style={'marginBottom': '10px', 'marginTop': '10px'}
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #Set up the features for your interactive data table to make it user-friendly for your client
        page_size = 10,  # limits the table to displaying 5 rows at once
        sort_action = "native",  # enables sorting
        filter_action = "native",  # enables filtering
        row_selectable = "single",  # only one row at a time can be selected
        selected_rows = [0]  # sets default selected row to the first row
    ),
    html.Br(),
    html.Hr(),
    html.Div([
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={
                'width': '45%',
                'height': '400px',
                'display': 'inline-block',
                'verticalAlign': 'top'
            }),
        html.Div(
            id='bar-chart',
            style={
                'width': '45%',
                'height': '400px',
                'display': 'inline-block',
                'verticalAlign': 'top'
            })
    # some flex box magic to get the chart and map side by side
    ], style={'display': 'flex', 'flexDirection': 'row','justifyContent': 'space-between', 
              'flexWrap': 'wrap', 'gap': '20px', 'padding': '10px'})
            
])

#############################################
# Interaction Between Components / Controller
#############################################
# Helper function that builds a query based on the filter selected from the radio buttons
def query_from_filter(rescue_type):
    if rescue_type == "water":
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == "mountain":
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == "tracking":
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        return {}  # reset

# This callback will filter the data table to only display data that meets the filter requirements
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-type', 'value')]
)
def filter_table(rescue_type):
    query = query_from_filter(rescue_type)
    filtered_results = shelter.read(query)
    return pd.DataFrame.from_records(filtered_results).drop(columns=['_id']).to_dict('records')   

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:  # prevents error on initial page load from no column being selected
        selected_columns = []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    #Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
              children=[
              dl.Tooltip(dff.iloc[row,4]),
              dl.Popup([
                 html.H1("Animal Name"),
                html.P(dff.iloc[row,9])
             ])
          ])
       ])
    ]

# This callback will update the bar chart showing outcome types after a filter has been applied
@app.callback(
    Output('bar-chart', 'children'),
    Input('datatable-id', 'derived_virtual_data')
)
def update_bar_chart(viewData):
    dff = pd.DataFrame.from_dict(viewData)

    fig = px.bar(
        dff['outcome_type'].value_counts().reset_index(),
        x='index',
        y='outcome_type',
        labels={'index': 'Outcome Type', 'outcome_type': 'Number of Dogs'},
        title='Outcomes for Selected Rescue Candidates'
    )

    return dcc.Graph(figure=fig)
    

app.run_server(debug=True)

Dash app running on http://127.0.0.1:27797/
